#### Documentação:
https://python.langchain.com/v0.2/docs/how_to/chatbots_memory/
https://python.langchain.com/docs/integrations/memory/
https://python.langchain.com/v0.2/docs/how_to/message_history/

### Definição

RouterChains de Langchain são estruturas que permitem o encadeamento de prompts de forma dinâmica, com base na entrada do usuário. Em outras palavras, elas funcionam como um sistema de triagem que decide qual será o próximo prompt utilizado, dependendo da pergunta anterior. Isso é especialmente útil quando você tem diferentes tipos de perguntas que precisam ser respondidas por diferentes modelos ou cadeias de prompts.

#### Por que usar RouterChains?
A principal vantagem de usar RouterChains é a capacidade de criar aplicações mais robustas e flexíveis. Imagine que você está desenvolvendo um chatbot para uma loja online. Com RouterChains, você pode direcionar perguntas sobre produtos eletrônicos para um especialista em eletrônicos, enquanto perguntas sobre roupas são direcionadas para um especialista em moda. Isso melhora a precisão e a relevância das respostas, proporcionando uma melhor experiência ao usuário.

In [ ]:
%run ../helpers/00-llm.ipynb

In [ ]:
from helpers.llm import initialize_llm, logger, pretty_print

llm, _, embeddings = initialize_llm()

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chain = (
    PromptTemplate.from_template(
        """
        Classifique a pergunta do usuário em uma das seguintes categorias:
        - Matemática
        - Física
        - Química
        - Outras Informações

        Pergunta: {query}
        Classificação:
        """
    )
    | llm
    | StrOutputParser() 
)
# Definindo os prompts especializados
chain_fisica = (
    PromptTemplate.from_template(
        """
        Você é um professor de física muito inteligente. Assuntos detalhados:
        - Mecânica: Estuda o movimento e as causas que o produzem ou alteram. Inclui cinemática (estudo do movimento sem considerar suas causas), dinâmica (estudo das forças que causam o movimento) e estática (estudo do equilíbrio de corpos). 
        - Termologia: Estuda o calor, a temperatura e suas relações. Abrange tópicos como calorimetria (estudo da troca de calor), termodinâmica (estudo das leis que regem as transformações de energia térmica), e estados físicos da matéria. 
        - Ondulatória: Estuda as ondas, fenômenos como a propagação de energia através de um meio ou do espaço. 
        - Óptica: Estuda a luz e seus fenômenos, como reflexão, refração, difração e interferência. 
        - Eletromagnetismo: Estuda os fenômenos relacionados à eletricidade e ao magnetismo, incluindo eletrostática (estudo das cargas elétricas em repouso), eletrodinâmica (estudo das cargas em movimento), e magnetismo. 
        - Pessoas influentes da area
        - Outras Informações

        Pergunta: {query} 
        Resposta:
        """
    )
    | llm
    | StrOutputParser() 
)
chain_matematica = (
    PromptTemplate.from_template(
        """
        Você é um professor de matemática muito inteligente. Assuntos detalhados:
        - Álgebra: Estuda as operações e relações entre números, incluindo equações, funções e polinômios.
        - Geometria: Estuda as propriedades e relações de pontos, linhas, superfícies e sólidos no espaço.
        - Trigonometria: Estuda as relações entre os ângulos e os lados dos triângulos.
        - Cálculo: Estuda as taxas de variação e as somas infinitas, incluindo derivadas e integrais.
        - Estatística: Estuda a coleta, análise e interpretação de dados numéricos.
        - Pessoas influentes da area
        - Outras Informações

        Pergunta: {query} 
        Resposta:
        """
    )
    | llm
    | StrOutputParser() 
)
chain_quimica = (
    PromptTemplate.from_template(
        """
        Você é um professor de química muito inteligente. Assuntos detalhados:
        - Estrutura Atômica: Estudo da composição do átomo, incluindo partículas subatômicas (prótons, nêutrons, elétrons), evolução dos modelos atômicos e distribuição eletrônica. 
        - Tabela Periódica: Organização dos elementos químicos, suas propriedades periódicas (eletroafinidade, energia de ionização, etc.) e relação com a estrutura atômica. 
        - Ligações Químicas: Tipos de ligações (iônica, covalente, metálica) e suas características, além de geometria molecular e polaridade. 
        - Funções Inorgânicas: Estudo de ácidos, bases, sais e óxidos, suas propriedades e reações. 
        - Reações Químicas: Tipos de reações, balanceamento de equações, reagente limitante, rendimento da reação e cálculo estequiométrico. 
        - Cálculos Químicos: Conceito de mol, número de Avogadro, massa molar, volume molar e cálculos relacionados a reações químicas. 
        - Termoquímica: Estudo do calor envolvido nas reações químicas, entalpia, lei de Hess e aplicações. 
        - Eletroquímica: Estudo da relação entre eletricidade e reações químicas, pilhas, eletrólise e potenciais de eletrodo. 
        - Química Orgânica: Estudo dos compostos de carbono, funções orgânicas, isomeria e reações orgânicas. 
        - Misturas e Separação de Misturas: Tipos de misturas (homogêneas e heterogêneas), métodos de separação (filtração, destilação, decantação, etc.). 
        - Equilíbrio Químico: Conceito de equilíbrio químico, constantes de equilíbrio, deslocamento do equilíbrio. 
        - Química Ambiental: Estudo da poluição ambiental, tratamento de resíduos e impacto das atividades humanas no meio ambiente. 
        - Radioatividade: Desintegração radioativa, tipos de radiação, meia-vida e aplicações da radioatividade.  
        - Pessoas influentes da area
        - Outras Informações

        Pergunta: {query} 
        Resposta:
        """
    )
    | llm
    | StrOutputParser() 
)

chain_prompt = (
    PromptTemplate.from_template(
        """
        Responda à seguinte pergunta:

        Pergunta: {query} 
        Resposta:
        """
    )
    | llm
    | StrOutputParser() 
)           


Um RunnableBranch é um tipo especial de executável que permite definir um conjunto de condições e executáveis a serem executados com base na entrada. Ele não oferece nada que você não possa obter em uma função personalizada:

In [ ]:
from langchain_core.runnables import RunnableBranch

branch = RunnableBranch(
    (lambda x: "física" in x["topic"].lower(), chain_fisica),
    (lambda x: "química" in x["topic"].lower(), chain_quimica),
    (lambda x: "matemática" in x["topic"].lower(), chain_matematica),
    chain_prompt,
)
 

full_chain = {"topic": chain, "query": lambda x: x["query"]} | branch
pretty_print(full_chain.invoke({"query": "Quanto é 2 +2?"}))



In [ ]:
from langchain_core.output_parsers import StrOutputParser

from langchain.prompts import PromptTemplate
 

# ela inicial (classificação)
chain = (
    PromptTemplate.from_template(
        """
        Classifique a pergunta do usuário em uma das seguintes categorias:
        - Assuntos Financeiros
        - Suporte Técnico
        - Atualização de Cadastro
        - Outras Informações

        Pergunta: {query}
        Classificação:
        """
    )
    | llm
    | StrOutputParser() 
)

# elos específicos
financial_chain = PromptTemplate.from_template(
    """
    Você é um especialista financeiro.
    Sempre responda às perguntas começando com "Bem-vindo ao Suporte Financeiro".
    Responda à pergunta do usuário:
    Pergunta: {query}
    Resposta:
    """
) | llm
tech_support_chain = PromptTemplate.from_template(
    """
    Você é um especialista em suporte técnico.
    Sempre responda às perguntas começando com "Bem-vindo ao Suporte Técnico".
    Ajude o usuário com seu problema técnico.
    Pergunta: {query}
    Resposta:
    """
) | llm
update_registration_chain = PromptTemplate.from_template(
    """
    Você é um representante de atendimento ao cliente.
    Sempre responda às perguntas começando com "Bem-vindo ao Suporte de Cadastro".
    Guie o usuário na atualização de suas informações de cadastro.
    Pergunta: {query}
    Resposta:
    """
) | llm
other_info_chain = PromptTemplate.from_template(
    """
    Você é um assistente de informações gerais.
    Sempre responda às perguntas começando com "Bem-vindo ao Suporte Geral".
    Forneça informações ao usuário sobre sua pergunta.
    Pergunta: {query}
    Resposta:
    """
) | llm
 
 
# Usando a RouterChain
#resposta = router_chain.run("Qual é a segunda Lei de Newton?")
#print(resposta)

In [ ]:
# Função de roteamento
def route(info):
    topic = info["topic"].lower()
    if "financeiro" in topic:
        return financial_chain
    elif "técnico" in topic:
        return tech_support_chain
    elif "atualização" in topic or "cadastro" in topic:
        return update_registration_chain
    else:
        return other_info_chain

In [ ]:
# Exemplos 1 suporte técnico
classification = chain.invoke({"query": "Como faço para redefinir minha senha?"})
print(classification)

In [ ]:
#chama a função rote, passando o topico
response_chain = route({"topic": classification})
print(response_chain)

In [ ]:
#executa o objeto correto
response = response_chain.invoke({"query": "Como faço para redefinir minha senha?"})
pretty_print(response)

In [ ]:
from langchain_core.runnables import RunnableLambda

router_chain = {"topic": chain, "query": lambda x: x["query"]} | RunnableLambda(
    route
)

query = "Qual é a missão da empresa?"
#query = "Como posso pagar uma fatura atrasada?"
#query = "Preciso alterar meu endereço de e-mail."

#query = "Como faço para redefinir minha senha?"

resposta = router_chain.invoke({"query": query})
pretty_print(resposta)

In [ ]:
from langchain_community.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

physics_template = """Você é um professor de física muito inteligente.
Você é ótimo em responder perguntas sobre física de forma concisa e fácil de entender.
Quando você não sabe a resposta para uma pergunta, você admite que não sabe. 

Aqui vai uma pergunta, seja curto na resposta:
{query}"""

math_template = """Você é um matemático muito bom. 
Você é ótimo em responder perguntas de matemática.
Você é tão bom porque consegue decompor problemas difíceis em suas partes componentes, responder às partes componentes e, em seguida, juntá-las para responder à questão mais ampla.

Aqui está uma pergunta, seja curto na resposta:
{query}"""

prompt_templates = [physics_template, math_template]
prompt_embeddings = embeddings.embed_documents(prompt_templates)
 
def prompt_router(input):
    query_embedding = embeddings.embed_query(input["query"])
    
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    
    most_similar = prompt_templates[similarity.argmax()]  
 
    print("Using MATH" if most_similar == math_template else "Using PHYSICS")
    return PromptTemplate.from_template(most_similar)


chain = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | llm
    | StrOutputParser()
)


In [ ]:
print(chain.invoke("What's a path integral?"))